In [ ]:
import os
import json
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
import plotly.express as px
import plotly.io as pio

# Jupyter Lab/Notebookでグラフを表示するための設定
pio.templates.default = "plotly_white"

In [ ]:
def process_file_for_mentions(filepath):
    """
    単一の.infoファイルを処理し,メンション情報のリストを返すワーカー関数
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        mentioner = data.get('owner', {}).get('username')
        timestamp = data.get('taken_at_timestamp')
        tagged_users = data.get('edge_media_to_tagged_user', {}).get('edges', [])
        
        if not mentioner or not timestamp or not tagged_users:
            return []
            
        mention_events = []
        for edge in tagged_users:
            mentioned_user = edge.get('node', {}).get('user', {}).get('username')
            if mentioned_user:
                mention_events.append({
                    'mentioner': mentioner,
                    'mentioned': mentioned_user,
                    'timestamp': timestamp
                })
        return mention_events
    except (json.JSONDecodeError, KeyError, FileNotFoundError):
        return [] # エラーが発生した場合は空リストを返す

def load_all_mention_data(info_dir):
    """
    全投稿の.infoファイルを並列処理で読み込み,メンションデータフレームを作成する
    """
    try:
        all_files = os.listdir(info_dir)
    except FileNotFoundError:
        print(f"エラー: ディレクトリ '{info_dir}' が見つかりません。")
        return pd.DataFrame()

    filepaths_to_process = [os.path.join(info_dir, f) for f in all_files if f.endswith('.info')]
    print(f"{len(filepaths_to_process)} 件の投稿ファイルを処理します...")

    all_mentions = []
    with ProcessPoolExecutor() as executor:
        future_to_filepath = {executor.submit(process_file_for_mentions, fp): fp for fp in filepaths_to_process}
        
        for future in tqdm(as_completed(future_to_filepath), total=len(filepaths_to_process), desc="メンション情報抽出中"):
            results = future.result()
            if results:
                all_mentions.extend(results)
    
    df = pd.DataFrame(all_mentions)
    if not df.empty:
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
        df = df.drop(columns=['timestamp'])
    return df

# 実行
INFO_DIR = 'posts_info/unzipped_data_7z/info/'
df_mentions = load_all_mention_data(INFO_DIR)

print(f"\n処理完了。合計 {len(df_mentions)} 件のメンション関係を抽出しました。")
df_mentions.head()

In [ ]:
target_user = "amandasperoni"

In [ ]:
if df_mentions.empty or target_user not in df_mentions['mentioned'].unique():
    print(f"エラー: '{target_user}' は一度もメンションされていないか,データが存在しませんでした。")
else:
    # 対象ユーザーがメンションされたデータのみを抽出
    df_user_indegree = df_mentions[df_mentions['mentioned'] == target_user].copy()
    
    # 日付をインデックスに設定
    df_user_indegree.set_index('datetime', inplace=True)
    
    # 週ごとにリサンプリングしてメンション数をカウント
    # 'W'は週末（日曜日）を基準に集計することを意味します
    weekly_indegree = df_user_indegree['mentioned'].resample('W').count()
    
    # グラフの作成
    fig = px.line(
        weekly_indegree,
        x=weekly_indegree.index,
        y='mentioned',
        markers=True,
        labels={'datetime': '日付', 'mentioned': '週ごとのメンション数'},
        title=f"'{target_user}' の週ごとの被メンション数（Indegree）推移"
    )
    
    fig.update_layout(
        xaxis_title='日付',
        yaxis_title='週ごとのメンション数',
        showlegend=False
    )
    
    fig.show()